Setup --program

In [8]:
import requests
from bs4 import BeautifulSoup
import time
import random

company_names = {}
all_jobs = []
jobs_details_list = []

def fetch_job_listings(page):
    url = f"https://www.dice.com/jobs?page={page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.find_all('div', class_='header -mb-2 flex flex-row justify-between')

def get_company_links(companies):
    for company in companies:
        name_company = company.find('p').text.strip()
        link = 'https://www.dice.com' + company.find('a').get('href')
        company_names[name_company] = link

In [9]:
def fetch_job_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    dice_id = soup.find('li', attrs={'data-testid': 'legalInfo-companyName'})
    company_name = soup.find('a', attrs={'data-wa-click': 'djv-job-company-profile-click'})
    skills = soup.find_all('li', class_='text-sm text-gray-700')
    job_name = soup.find('h1', class_='text-pretty font-bold text-2xl sm:text-2xl md:text-2xl lg:text-3xl xl:text-3xl 2xl:text-3xl')
    posted_element = soup.find_all('div', {"class": 'mt-4'})
    start_li = soup.find_all('span', {'class': 'text-sm font-normal text-font-light'})
    if start_li:
        posted_date = start_li[0].find_all('span')[1].get_text(strip=True)
        clean_date = posted_date.replace('• Posted', '').strip()
    
    
    return {
        "post_id": dice_id.text.split()[2] if dice_id else None,
        "company_name": company_name.get_text(strip=True) if company_name else None,
        "job_name": job_name.get_text(strip=True) if job_name else None,
        "skills": [skill.get_text(strip=True) for skill in skills] if skills else [],
        "date_scraped": time.strftime("%Y-%m-%d"),
        "date_posted": clean_date
    }


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [11]:
def get_company_job_links(company_url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    all_job_links = []
    
    try:
        # เริ่มต้นที่หน้าแรกก่อนเพื่อเช็คจำนวนหน้าทั้งหมด
        driver.get(company_url)
        wait = WebDriverWait(driver, 15)
        
        # 1. เข้าไปที่ Jobs Tab เพื่อให้ Shadow DOM ของรายการงานโหลด
        jobs_tab = wait.until(EC.element_to_be_clickable((By.XPATH, "//li[contains(text(), 'Jobs')]")))
        jobs_tab.click()
        
        # รอให้รายการงานปรากฏ
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "dhi-job-search-job-list")))
        time.sleep(4) 

        def get_total_pages():
            script = """
            const pagination = document.querySelector('dhi-job-search-job-list')?.shadowRoot
                               ?.querySelector('seds-pagination')?.shadowRoot.querySelector('.page');
            if (!pagination) return 1;
            const spans = pagination.querySelectorAll('span');
            for (let s of spans) {
                if (s.innerText.includes('of')) {
                    return parseInt(s.innerText.replace(/[^0-9]/g, ''));
                }
            }
            return 1;
            """
            return int(driver.execute_script(script))

        total_pages = get_total_pages()
        #print(f"Detected Total Pages: {total_pages}")

        # สคริปต์ดึง Link (เจาะ Shadow DOM)
        pierce_links_script = """
        function findAllLinksDeep(selector, root = document, links = []) {
            const foundInRoot = root.querySelectorAll(selector);
            foundInRoot.forEach(el => { if (el.href) links.push(el.href); });
            const hosts = root.querySelectorAll('*');
            for (const host of hosts) {
                if (host.shadowRoot) findAllLinksDeep(selector, host.shadowRoot, links);
            }
            return links;
        }
        return findAllLinksDeep('a.job-title-link');
        """

        for page_num in range(1, total_pages + 1):
            # สร้าง URL สำหรับแต่ละหน้า
            # ตรวจสอบว่ามี ? อยู่ใน URL เดิมหรือยังเพื่อเลือกใช้ & หรือ ?
            separator = "&" if "?" in company_url else "?"
            page_url = f"{company_url}{separator}page={page_num}"
            
            #print(f"Scraping page {page_num}: {page_url}")
            driver.get(page_url)
            
            # หลังจากเปลี่ยน URL ต้องรอให้ Shadow DOM โหลดใหม่
            time.sleep(5) 
            
            # ดึงข้อมูล Link
            current_links = driver.execute_script(pierce_links_script)
            
            initial_count = len(all_job_links)
            for link in current_links:
                if link not in all_job_links:
                    all_job_links.append(link)
            
            #print(f"Added {len(all_job_links) - initial_count} new links.")

        #print(f"Extraction complete. Found {len(all_job_links)} total links.")
        
    finally:
        driver.quit()
    return all_job_links

In [12]:
import pickle
def save_fetched_data(data, filename='jobs_data.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(data, f) 
    print(f"Data saved to {filename}")  

In [13]:
import json
import os

def save_all_to_json(all_scraped_data, filename='jobs_data.json'):
    """
    บันทึกข้อมูลลงในโฟลเดอร์ Raw_data โดยถอยออกจากโฟลเดอร์ src
    """
    if not all_scraped_data:
        print("ไม่มีข้อมูลให้บันทึก")
        return

    # 1. กำหนด Path โดยถอยออกจาก src 1 ชั้นเพื่อเข้าสู่ Raw_data
    # .. หมายถึงถอยหลัง 1 ก้าว
    target_folder = os.path.join('..', 'Raw_data')
    full_path = os.path.join(target_folder, filename)

    # 2. ตรวจสอบและสร้างโฟลเดอร์ Raw_data หากยังไม่มี (กันเหนียว)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        print(f"สร้างโฟลเดอร์ใหม่ที่: {target_folder}")

    # 3. โหลดข้อมูลเดิมที่มีอยู่ในไฟล์ (ถ้ามี)
    master_data = []
    if os.path.exists(full_path):
        with open(full_path, 'r', encoding='utf-8') as f:
            try:
                master_data = json.load(f)
            except (json.JSONDecodeError, FileNotFoundError):
                master_data = []

    # 4. เตรียมข้อมูลใหม่ (Clean ข้อมูลเหมือนเดิม)
    new_entries = []
    for entry in all_scraped_data:
        clean_entry = {
            "post_id": str(entry.get('post_id') or ""),
            "company_name": str(entry.get('company_name') or ""),
            "job_name": str(entry.get('job_name') or ""),
            "skills": [str(s) for s in entry.get('skills', [])],
            "date_scraped": str(entry.get('date_scraped') or ""),
            "date_posted": str(entry.get('date_posted') or "").replace('•', '').replace('Posted', '').strip()
            
        }
        new_entries.append(clean_entry)

    # 5. รวมข้อมูลและบันทึกลงไปใน Raw_data
    master_data.extend(new_entries)
    with open(full_path, 'w', encoding='utf-8') as f:
        json.dump(master_data, f, ensure_ascii=False, indent=4)
    
    print(f"--- บันทึกสำเร็จ! ---")
    print(f"ตำแหน่งไฟล์: {os.path.abspath(full_path)}") # โชว์ Path เต็มๆ เพื่อความมั่นใจ
    print(f"เพิ่ม {len(new_entries)} รายการ (รวมทั้งหมด {len(master_data)})")

In [14]:
for page in range(1, 26):
    companies = fetch_job_listings(page)
    print(f"Page {page} companies successfully fetched.")
    time.sleep(random.uniform(1, 3))
    get_company_links(companies)
print(f'Number of company links = {len(company_names)}')

Page 1 companies successfully fetched.
Page 2 companies successfully fetched.
Page 3 companies successfully fetched.
Page 4 companies successfully fetched.
Page 5 companies successfully fetched.
Page 6 companies successfully fetched.
Page 7 companies successfully fetched.
Page 8 companies successfully fetched.
Page 9 companies successfully fetched.
Page 10 companies successfully fetched.
Page 11 companies successfully fetched.
Page 12 companies successfully fetched.
Page 13 companies successfully fetched.
Page 14 companies successfully fetched.
Page 15 companies successfully fetched.
Page 16 companies successfully fetched.
Page 17 companies successfully fetched.
Page 18 companies successfully fetched.
Page 19 companies successfully fetched.
Page 20 companies successfully fetched.
Page 21 companies successfully fetched.
Page 22 companies successfully fetched.
Page 23 companies successfully fetched.
Page 24 companies successfully fetched.
Page 25 companies successfully fetched.
Number of

Selenium

In [15]:
selenium_start = 0
selenium_end = len(company_names)
for company_url in list(company_names.values())[selenium_start:selenium_end]:
    try:
        jobs_link = get_company_job_links(company_url)
        all_jobs.extend(jobs_link)
        print(f"Fetched job links from {len(jobs_link)} jobs at {company_url} Successfully.")
        time.sleep(random.uniform(1, 3))
    except Exception as e:
        print(f"An error occurred during Selenium scraping: {e}")
    finally:
        continue
print(f'Total job links fetched: {len(all_jobs)}')
save_fetched_data(all_jobs)

Fetched job links from 114 jobs at https://www.dice.com/company-profile/fc306a81-f888-56a5-9562-e823c5922cf6?companyname=Trans%20Union%20LLC Successfully.
Fetched job links from 0 jobs at https://www.dice.com/company-profile/bbe7fc64-a1a8-4797-8e30-480f21f94ce0?companyname=Belcan%2C%20LLC Successfully.
Fetched job links from 153 jobs at https://www.dice.com/company-profile/3569f9d0-5cbb-573c-ab60-cd2c3547f2dd?companyname=Kratos%20Defense%20and%20Security%20Solutions%2C%20Inc. Successfully.
Fetched job links from 202 jobs at https://www.dice.com/company-profile/167507f4-0f08-514a-b36c-968e72eaa9ce?companyname=GovCIO Successfully.
Fetched job links from 72 jobs at https://www.dice.com/company-profile/a573ffe3-0826-5bd1-bc94-d8da46a5cc72?companyname=Medline%20Industries%2C%20LP Successfully.
Fetched job links from 14 jobs at https://www.dice.com/company-profile/2db0dd0c-2773-49f9-a6af-fbf62bbd40ca?companyname=American%20IT%20Systems Successfully.
Fetched job links from 473 jobs at https:/

Jobs Details

In [16]:
start = 0
end = len(all_jobs)
fetching_error = []
for i, job_url in enumerate(all_jobs[start:end]):  # Fetch details for first 500 jobs
    try:
        print(f"Fetching details for URL: {job_url}")
        job_details = fetch_job_details(job_url)
        if job_details not in jobs_details_list:
            jobs_details_list.append(job_details)
        print(f'{i+1}. fetched successfully.')
        time.sleep(random.uniform(1, 3))
    except Exception as e:
        print(f'Error fetching job details from {job_url}: {e}')
        fetching_error.append(job_url)
    finally:
        continue
print(f'Total job details fetched: {len(jobs_details_list)}')
save_fetched_data(jobs_details_list, filename='jobs_details_data.pkl')
save_fetched_data(fetching_error, filename='fetching_error.pkl')

Fetching details for URL: https://www.dice.com/job-detail/a0dc6208-9928-496e-9bf8-a8956ed62eb2
1. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/3d28b483-3396-4059-a162-107c3ded0ba1
2. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/10cdd18a-a821-4432-93a4-40a8f4798190
3. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/4a054b81-86ff-437a-a43e-11495c3c4260
4. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/8ee0561d-91d3-4c68-b03c-3873bcf11e96
5. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/1d0b0167-5de1-4405-ad4a-9ec730eb30ad
6. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/61eaabf0-da5e-4398-bf03-ca7f708a9e46
7. fetched successfully.
Fetching details for URL: https://www.dice.com/job-detail/1de0b9c3-f7ad-4216-94c9-250e6cec7ccd
8. fetched successfully.
Fetching details for URL: https://www.di

เก็บไว้ใน JSON

In [17]:
save_all_to_json(jobs_details_list, filename='jobs_data2.json')

--- บันทึกสำเร็จ! ---
ตำแหน่งไฟล์: c:\GitHub\Clone\programming-language-trends-analysis\Raw_data\jobs_data2.json
เพิ่ม 2633 รายการ (รวมทั้งหมด 2633)
